# 01_scrap

- scrapping data from eventseye.com and upload to DataPlatform

# Getting data from eventseye.com

In [ ]:
pip install scrapy

# Import Library

In [ ]:
import scrapy
import pandas as pd
import json
import csv


from scrapy.crawler import CrawlerProcess
from scrapy.crawler import CrawlerRunner

In [ ]:
class Eventseye(scrapy.Spider):
    name = "eventseye"
    start_urls = [
        'https://www.eventseye.com/fairs/st1_trade-shows_jewelry.html',
        'https://www.eventseye.com/fairs/st1_trade-shows_jewelry_1.html',
        'https://www.eventseye.com/fairs/st1_trade-shows_jewelry_2.html',
        'https://www.eventseye.com/fairs/st1_trade-shows_jewelry_3.html'
    ]
    custom_settings = {
        'FEEDS': { 'eventseye3.json': { 'format': 'json',}}
        }
        
    def parse(self, response):
        for event in response.css('tr'):
            yield {
                'Exhibition_Name': event.css('b::text').get(),
                'Describe': event.css('i::text').get(),
                'Venue': event.css('a.city::text').get(),
                'Place': event.css('a.place::text').get(),

            }

In [ ]:
process = CrawlerProcess()
process.crawl(Eventseye)
process.start()

- Clean Data

In [ ]:
df=pd.read_json('eventseye3.json')
df

In [ ]:
df.isnull()

In [ ]:
df = df.dropna()
df.isnull().sum()

In [ ]:
def split_venue(data):
  data = data.split("(")
  return data

In [ ]:
df[['Country', 'Venue_City',]] = list(map(split_venue, df['Venue']))
df['Country'] = df['Country'].astype(str)
df['Venue_City'] = df['Venue_City'].astype(str)

In [ ]:
df=df.drop(columns=['Venue'])

In [ ]:
df

In [ ]:
def split_city(data):
  data = data.split(")")
  return data

In [ ]:
df[['City','Pass']] = list(map(split_city, df['Venue_City']))
df['City'] = df['City'].astype(str)

df=df.drop(columns=['Venue_City','Pass'])

In [ ]:
df

In [ ]:
eventseye = df

In [ ]:
print(f"Total events : {len(eventseye)}")

In [ ]:
eventseye = eventseye.to_dict('records')

- dump data to json file

In [ ]:
with open('eventseye.json', 'w') as f:
    json.dump(eventseye, f)

# Upload data to DataPlatform

In [ ]:
# import pandas as pd
# df = pd.DataFrame(columns=["Exhibition_Name", "Describe","Country","City", "Place"])

In [ ]:
import requests
from tqdm.auto import tqdm

In [ ]:
from dsmlibrary.datanode import DataNode

In [ ]:
dir_raw_id = 62

In [ ]:
datanode = DataNode()

In [ ]:
datanode.upload_file(directory_id=dir_raw_id, file_path='eventseye.json')